<a href="https://colab.research.google.com/github/Azure-Vision/NovelAI/blob/master/NovelAILeaks_API_Backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

基于 4chan 魔改版 NovelAILeaks (naifu) 制作。[来源](https://boards.4channel.org/g/thread/89095460#p89097704)

使用官方前端 + 优化版后端，可突破75限制，支持所有模型。

Credit: https://t.me/StableDiffusion_CN https://t.me/exlolicon

Thanks: Anonymous, 炼铜术士, 神楽坂早苗️, Jonathan, 咕 咕, 猫又逆变器, Gaein nidb



In [3]:
!nvidia-smi

Mon Oct 31 13:23:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.64       Driver Version: 430.64       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000001:00:00.0 Off |                    0 |
| N/A   32C    P0    37W / 250W |   3785MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000002:00:00.0 Off |                    0 |
| N/A   31C    P0    36W / 250W |   4122MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [8]:
#@title ### 1. 下载 Novel AI API 后端、模型 
#@markdown 如果下载速度太慢可尝试 restart

%cd content/
!apt install -y -qq aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar 
  
!echo "Decompressing..."
!tar xf naifu.tar && rm naifu.tar
!echo "Done."

[Errno 2] No such file or directory: 'content/'
/home/TableSense/largedisk/wanrong/NovelAI/content

10/31 13:26:12 [ERROR] CUID#7 - Download aborted. URI=https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar
Exception: [AbstractCommand.cc:350] errorCode=1 URI=https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar
  -> [SocketCore.cc:975] errorCode=1 SSL/TLS handshake failure:  `not signed by known authorities or invalid' `expired'

10/31 13:26:12 [NOTICE] Download GID#6c9db2f58fa0f9de not complete: 

10/31 13:26:12 [ERROR] CUID#6 - Download aborted. URI=https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar
Exception: [AbstractCommand.cc:350] errorCode=1 URI=https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar
  -> [SocketCore.cc:975] errorCode=1 SSL/TLS handshake failure:  `not signed by known authorities or invalid' `expired'

10/31 13:26:12 [NOTICE] Download GID#6322b43c0d76bea6 not complete: 

Download Results:
gid   |stat|avg sp

In [10]:
#@title ### 2. 安装依赖
#@markdown 耐心等待安装完成

%cd content/naifu
!pip install virtualenv && bash ./setup.sh
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx # -C /usr/bin
# !curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared
!curl -Lo ./cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x ./cloudflared

[Errno 2] No such file or directory: 'content/naifu'
/home/TableSense/largedisk/wanrong/NovelAI/content
  Using cached virtualenv-20.16.6-py3-none-any.whl (8.8 MB)
  Using cached importlib_resources-5.4.0-py3-none-any.whl (28 kB)
  Using cached distlib-0.3.6-py2.py3-none-any.whl (468 kB)
  Using cached platformdirs-2.4.0-py3-none-any.whl (14 kB)
  Using cached importlib_metadata-4.8.3-py3-none-any.whl (17 kB)
  Using cached filelock-3.4.1-py3-none-any.whl (9.9 kB)
  Attempting uninstall: importlib-resources
    Found existing installation: importlib-resources 3.3.0
    Uninstalling importlib-resources-3.3.0:
      Successfully uninstalled importlib-resources-3.3.0
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 2.0.0
    Uninstalling importlib-metadata-2.0.0:
      Successfully uninstalled importlib-metadata-2.0.0
  Attempting uninstall: filelock
    Found existing installation: filelock 3.4.0
    Uninstalling filelock-3.4.0:
      Success

In [ ]:
#@title ### 3. 启动模型
#@markdown 访问输出的映射地址（以 `trycloudflare.com` / `bore.pub` 结尾）即可。
#@markdown - 请等待模型加载完成（出现`Application startup complete`字样）再访问
#@markdown - cloudflare 提供的服务偶尔会出现请求超时，可换用 bore 隧道

%cd content/naifu
!sed -i 's/# export SAVE_FILES="1"/export SAVE_FILES="1"/g' run.sh
!bash run.sh # & ./cloudflared tunnel --url localhost:6969 

In [ ]:
#@title ### 4. （可选）使用 7G 的 `animefull-latest` 模型运行
#@markdown 默认使用的是 4G 大小的 animefull-final-pruned 模型。如果想使用 7G 的 animefull-latest 模型，运行这个

%cd /content/
!tar xf animefull-latest.tar -C /content/naifu/models && rm animefull-latest.tar
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/naifu/hydra_node/models.py

%cd /content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
%env MODEL_PATH=models/animefull-latest
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 0.0.0.0 --port=6969 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969 